In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_10월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(1113269, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
1113269


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0,"[10월, 1일, 날, 갔던, 캠핑, 인데, 포스팅, 을, 미뤘더니, 깜빡, 해, ...","[('10월', 'Number'), ('1일', 'Number'), ('날', 'N..."
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0,"[이, 날, 날씨, 가, 너무, 좋다, 보니, 캠핑장, 과, 펜션, 은, 당연히, ...","[('이', 'Determiner'), ('날', 'Noun'), ('날씨', 'N..."
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0,"[친한, 언니, 들, A, 언니, 와, B, 언니, 부부, 와, 함께, 갔던, 날,...","[('친하다', 'Adjective'), ('언니', 'Noun'), ('들', '..."
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0,"[푸릇푸릇, 한, 밤나무, 들, 이, 인상, 적, 이다, 주차장, 에도, 차가, 정...","[('푸릇푸릇', 'Noun'), ('하다', 'Verb'), ('밤나무', 'No..."
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0,"[차가, 많이, 막혀서, 일찍, 출발, 했는데도, 시간, 이, 꽤, 걸렸다]","[('차갑다', 'Adjective'), ('많이', 'Adverb'), ('막히다..."
...,...,...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375,"[관람, 을, 마치, 고, 나오면, 야외, 에도, 이렇게, 좌, 불상, 들, 이, 있다]","[('관람', 'Noun'), ('을', 'Josa'), ('마치', 'Noun')..."
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375,"[진짜, 중국, 스케일, 최고, 불교, 는, 아니지만, 중국, 도시, 를, 여행, ...","[('진짜', 'Noun'), ('중국', 'Noun'), ('스케일', 'Noun..."
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375,"[뭔가, 현, 실세, 계, 가, 아닌, 느낌, 이, 랄, 까]","[('뭔가', 'Noun'), ('현', 'Modifier'), ('실세', 'No..."
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375,"[그래서, 세계, 인, 이, 주목, 하는, 미래, 형, 불교, 사찰, 인거, 겠지,...","[('그래서', 'Adverb'), ('세계', 'Noun'), ('인', 'Suf..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(165188, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '때', '등', '우리', '말', '문화', '더', '생각', '나', '내', '길', '시간', '사진', '나무', '위', '의', '전', '공원', '여행', '안', '제', '중', '때문', '마을', '하나', '일', '가을', '아이', '집', '날', '체험', '저', '시작', '개', '위해', '공간', '다른', '모습', '지역', '작품', '한국', '앞', '또', '마음', '거']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 1113269/1113269 [00:08<00:00, 134513.39it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

16
16


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 1113269/1113269 [12:52<00:00, 1441.85it/s]


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/10월/단어동시출현_딕셔너리_10월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 183542573/183542573 [21:38<00:00, 141305.95it/s]


(183542573, 3)


,term1,term2,freq
0,다,중이,1027
1,날,중이,66
2,1일,중이,12
3,을,중이,1143
4,중이,포스팅,17


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/10월/단어동시출현_df_10월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 183542573/183542573 [37:20<00:00, 81937.30it/s]


,term1,term2,freq
0,날,이,18043
1,수,이,77484
2,날,수,3464
3,날,우리,1846
4,말,이,30132
...,...,...,...
1220,때문,여행,679
1221,때문,앞,735
1222,아이,앞,686
1223,개,아이,554


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/10월/단어동시출현_df_상위50_10월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1530.48it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,때,등,우리,말,문화,더,생각,나,내,길,시간,사진,나무,위,의,전,공원,여행,안,제,중,때문,마을,하나,일,가을,아이,집,날,체험,저,시작,개,위해,공간,다른,모습,지역,작품,한국,앞,또,마음,거
것,0.0,32462.0,30965.0,118890.0,11555.0,18742.0,15413.0,8854.0,13951.0,15014.0,6950.0,11852.0,13569.0,15913.0,12352.0,5103.0,7358.0,4550.0,5890.0,8556.0,88877.0,7417.0,2095.0,3554.0,8915.0,6048.0,7751.0,10161.0,2395.0,6811.0,9977.0,2896.0,4549.0,4458.0,4471.0,2322.0,6319.0,5041.0,3753.0,6429.0,3409.0,7474.0,4523.0,4051.0,3593.0,3457.0,4970.0,5331.0,6294.0,3862.0
수,32462.0,0.0,18368.0,77484.0,14609.0,11986.0,9691.0,11553.0,9030.0,8412.0,8221.0,9019.0,8199.0,11299.0,8387.0,6028.0,8014.0,5211.0,5850.0,6432.0,67705.0,6518.0,4326.0,4338.0,6200.0,5818.0,6340.0,7114.0,3265.0,4508.0,7070.0,4450.0,4920.0,3488.0,3464.0,5753.0,4166.0,3553.0,4047.0,4317.0,6493.0,5184.0,4753.0,3928.0,4073.0,3172.0,4025.0,3748.0,4208.0,2858.0
그,30965.0,18368.0,0.0,66366.0,9903.0,13211.0,10676.0,5861.0,8165.0,9520.0,4174.0,6088.0,6862.0,9912.0,7635.0,3718.0,4102.0,2551.0,3804.0,6036.0,63906.0,5271.0,1308.0,1870.0,5773.0,3971.0,7124.0,6186.0,1928.0,4249.0,5979.0,1464.0,2276.0,3245.0,3206.0,1120.0,3255.0,3505.0,2581.0,4325.0,2132.0,4354.0,3639.0,2619.0,2992.0,2152.0,3544.0,3568.0,3821.0,2596.0
이,118890.0,77484.0,66366.0,0.0,53730.0,48933.0,39646.0,31889.0,28754.0,30132.0,21309.0,27350.0,29164.0,37668.0,28784.0,19489.0,24148.0,18742.0,20071.0,20563.0,227388.0,22539.0,11104.0,11985.0,24488.0,19041.0,21972.0,21429.0,11732.0,15817.0,21418.0,13723.0,14685.0,14501.0,18043.0,9898.0,17714.0,15526.0,13680.0,14810.0,14260.0,16060.0,15561.0,12527.0,12126.0,10884.0,14329.0,13636.0,15328.0,13919.0
곳,11555.0,14609.0,9903.0,53730.0,0.0,5833.0,5349.0,4997.0,3323.0,2674.0,4658.0,3917.0,3590.0,5069.0,3853.0,5082.0,4119.0,4234.0,3460.0,2926.0,30789.0,3660.0,4081.0,4968.0,3636.0,3529.0,4625.0,2375.0,3666.0,2534.0,2535.0,4116.0,3484.0,2472.0,2207.0,2841.0,3319.0,2057.0,2142.0,2207.0,3298.0,3054.0,2553.0,2480.0,1166.0,1683.0,2355.0,2034.0,2072.0,1430.0
사람,18742.0,11986.0,13211.0,48933.0,5833.0,0.0,6323.0,3176.0,4985.0,6219.0,2925.0,4379.0,5448.0,7413.0,5088.0,2733.0,3080.0,2751.0,2507.0,3111.0,32616.0,2845.0,1127.0,1328.0,4133.0,2103.0,3220.0,3633.0,1651.0,2495.0,4113.0,1124.0,1391.0,2224.0,2210.0,736.0,2550.0,2121.0,1421.0,2880.0,1537.0,4306.0,2177.0,1563.0,1158.0,1454.0,2371.0,2203.0,2909.0,2056.0
때,15413.0,9691.0,10676.0,39646.0,5349.0,6323.0,0.0,3187.0,4483.0,4718.0,2031.0,3806.0,4481.0,6327.0,5136.0,2358.0,3059.0,2310.0,2197.0,2593.0,26813.0,3227.0,937.0,1823.0,3761.0,2882.0,2914.0,2732.0,1179.0,2268.0,3499.0,1261.0,1833.0,2505.0,2352.0,746.0,2591.0,2355.0,1492.0,2057.0,1099.0,2236.0,1827.0,1303.0,956.0,1257.0,2035.0,2044.0,2241.0,2058.0
등,8854.0,11553.0,5861.0,31889.0,4997.0,3176.0,3187.0,0.0,2022.0,2442.0,5211.0,1993.0,1668.0,4433.0,2830.0,2056.0,2249.0,1706.0,3145.0,3401.0,31191.0,3374.0,2185.0,1351.0,2017.0,3268.0,3020.0,1691.0,1829.0,1327.0,2630.0,1454.0,1504.0,1500.0,1149.0,3070.0,1127.0,1640.0,2757.0,2268.0,2400.0,1552.0,1592.0,2830.0,2195.0,2210.0,1368.0,1039.0,874.0,591.0
우리,13951.0,9030.0,8165.0,28754.0,3323.0,4985.0,4483.0,2022.0,0.0,4072.0,2657.0,3221.0,3905.0,4372.0,3273.0,1963.0,2708.0,1625.0,1767.0,2447.0,25743.0,2274.0,672.0,1322.0,3075.0,1489.0,2112.0,2430.0,869.0,1948.0,2883.0,911.0,1913.0,2615.0,1846.0,949.0,1498.0,1494.0,1078.0,2063.0,1017.0,2035.0,1444.0,680.0,984.0,1183.0,1826.0,1783.0,1923.0,1527.0
말,15014.0,8412.0,9520.0,30132.0,2674.0,6219.0,4718.0,2442.0,4072.0,0.0,1695.0,2830.0,3418.0,5268.0,4367.0,1382.0,1670.0,914.0,1467.0,2166.0,24319.0,2223.0,439.0,730.0,3044.0,1840.0,2070.0,2224.0,642.0,1732.0,2798.0,718.0,1270.0,1284.0,1287.0,476.0,1677.0,1329.0,1123.0,1626.0,569.0,1894.0,1129.0,980.0,712.0,983.0,1555.0,1547.0,1888.0,1534.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/10월/공출현빈도_df_10월.csv", encoding='UTF-8')